# Async Functionality in Python

#### We want to have coffee and bagel in the morning after waking up

## Synchronous or Normal Version


In [2]:
import time
def brew_coffee():
    print("Brewing Coffee")
    time.sleep(3) # 3 Minutes
    print("Coffee Ready")

def toast_bagel():
    print("Toasting Bagel")
    time.sleep(2) # 2 Minutes
    print("Bagel Ready")


def main():
    start = time.time()

    coffee = brew_coffee()
    # time.sleep(2)
    bagel = toast_bagel()

    end = time.time()


    print(f"Time : {end - start:.2f} seconds")

main()

Brewing Coffee
Coffee Ready
Toasting Bagel
Bagel Ready
Time : 5.01 seconds


## Asynchronous Version

In [3]:
import asyncio
import time

async def brew_coffee():
    print("Starting Brewing Coffee")
    await asyncio.sleep(3)
    print("Coffee Ready")

async def toast_bagel():
    print("Start Toasting bagel")
    await asyncio.sleep(2)
    print("Bagel Ready")


async def main():
    start = time.time()

    coffee = brew_coffee()
    bagel = toast_bagel()

    results = await asyncio.gather(coffee,bagel)

    end = time.time()

    print(f"Time : {end - start:.2f} seconds")


await main()

Starting Brewing Coffee
Start Toasting bagel
Bagel Ready
Coffee Ready
Time : 3.00 seconds


## With Tasks

In [4]:
import asyncio
import time

async def brew_coffee_async():
    print("Starting Brewing Coffee")
    await asyncio.sleep(3)
    print("Coffee Ready")

async def toast_bagel_async():
    print("Start Toasting bagel")
    await asyncio.sleep(2)
    print("Bagel Ready")

async def main_individual():
    start = time.time()
    coffee_task = asyncio.create_task(brew_coffee_async())
    bagel_task = asyncio.create_task(toast_bagel_async())

    coffee = await coffee_task
    bagel = await bagel_task

    end = time.time()
    print(f"Time : {end - start:.2f} minutes")


await (main_individual())



Starting Brewing Coffee
Start Toasting bagel
Bagel Ready
Coffee Ready
Time : 3.00 minutes


## **1. Subroutine**

* **Definition:**
  A subroutine (function or procedure) is the simplest unit of reusable code. You call it, it runs top to bottom, and when it’s done, control returns to the caller.
* **Python example:**

  ```python
  def greet(name):
      return f"Hello, {name}!"

  print(greet("Suraj"))
  ```
* **Key traits:** Linear execution, always starts at the top and exits at the bottom.
* **Pros:**

  * Clear, simple, reusable.
  * Easy to debug.
* **Cons:**

  * Blocking: can’t pause/resume in the middle.
  * Limited flexibility for tasks that need interruptions (e.g., waiting for I/O).

---

## **2. Coroutine**

* **Definition:**
  A coroutine is like a subroutine that can *pause* and *resume*. Instead of just returning once, it can yield control back and pick up later where it left off.
* **Python example:**

  ```python
  def countdown(n):
      while n > 0:
          yield n
          n -= 1

  for num in countdown(3):
      print(num)
  ```

  Or with `async def` in modern Python:

  ```python
  import asyncio

  async def greet():
      print("Hello")
      await asyncio.sleep(1)
      print("World")
  ```
* **Key traits:** Cooperative multitasking; execution pauses at `yield` or `await`.
* **Pros:**

  * Good for I/O-bound tasks (networking, file I/O).
  * Lightweight compared to threads.
* **Cons:**

  * Requires explicit `await` (programmer responsibility).
  * Not truly parallel (only one coroutine runs at a time on a single thread).

---

## **3. Synchronous Programming**

* **Definition:**
  Tasks run one after the other, in a blocking fashion. Each step waits for the previous one to complete.
* **Python example:**

  ```python
  import time

  def task():
      time.sleep(2)
      print("Task done")

  task()
  print("Next line")
  ```
* **Pros:**

  * Simple, predictable.
  * Easier debugging.
* **Cons:**

  * Inefficient for I/O-heavy programs (CPU sits idle while waiting).
  * Doesn’t scale well for high concurrency.

---

## **4. Asynchronous Programming**

* **Definition:**
  Code doesn’t block; tasks can yield control while waiting, so other tasks can run in the meantime.
* **Python example with `asyncio`:**

  ```python
  import asyncio

  async def task(name):
      print(f"Start {name}")
      await asyncio.sleep(2)
      print(f"End {name}")

  asyncio.run(asyncio.gather(task("A"), task("B")))
  ```
* **Pros:**

  * Efficient for I/O-bound workloads (web servers, APIs, scrapers).
  * Many tasks can progress “at once” without threads.
* **Cons:**

  * Harder mental model.
  * Debugging is trickier.
  * Doesn’t help with CPU-bound tasks.

---

## **5. Concurrency**

* **Definition:**
  Structuring a program so multiple tasks *make progress* at the same time. They *may not* literally run simultaneously but are interleaved.
* **Python examples:**

  * **With asyncio (single thread):**
    Good for thousands of network connections.
  * **With threading:**
    Several threads interleave execution.
* **Pros:**

  * Great for I/O-bound workloads.
  * Improves responsiveness.
* **Cons:**

  * More complex logic (synchronization issues with threads).
  * Threads in Python are limited by the GIL for CPU-bound tasks.

---

## **6. Parallelism**

* **Definition:**
  Actual simultaneous execution on multiple CPU cores or machines.
* **Python example (multiprocessing):**

  ```python
  from multiprocessing import Pool

  def square(n):
      return n*n

  with Pool(4) as p:
      print(p.map(square, [1,2,3,4]))
  ```
* **Pros:**

  * True speedup for CPU-bound tasks (math, ML training).
  * Can use all cores.
* **Cons:**

  * More overhead (processes heavier than threads).
  * Data sharing is tricky (need IPC, serialization).
  * Not always worth it for small workloads.

---

## **Putting It Together: Key Differences**

| Concept          | Runs How?                             | Pausable?       | Suitable for                      | Python Tooling                   |
| ---------------- | ------------------------------------- | --------------- | --------------------------------- | -------------------------------- |
| **Subroutine**   | Sequential                            | ❌               | General tasks                     | Functions                        |
| **Coroutine**    | Sequential but pausable               | ✅               | I/O tasks, cooperative scheduling | `async def`, `await`, generators |
| **Synchronous**  | One after another                     | ❌               | Simple scripts, CPU tasks         | Normal Python code               |
| **Asynchronous** | Interleaved                           | ✅               | I/O-bound (servers, APIs)         | `asyncio`, `aiohttp`             |
| **Concurrency**  | Many tasks making progress            | Depends         | I/O-bound, responsiveness         | `asyncio`, `threading`           |
| **Parallelism**  | Tasks *actually* run at the same time | ❌ (independent) | CPU-heavy tasks                   | `multiprocessing`, `joblib`      |

---

👉 **Rule of thumb:**

* **Use subroutines** for everyday programming.
* **Use coroutines / async** when you’re waiting on *lots of I/O*.
* **Use concurrency** to keep many tasks “in flight.”
* **Use parallelism** to chew through heavy CPU work on multiple cores.

